In [ ]:
# https://github.com/johanna23cct/integrated-CA2-MSc-2023094.git

In [ ]:
#git remote add origin https://github.com/johanna23cct/integrated-CA2-MSc-2023094.git
#git branch -M main
#git push -u origin main

In [ ]:
pip install Matplotlib

In [ ]:
pip install bokeh

In [ ]:
pip install skforecast --user

In [1]:
# Data manipulation
# ==============================================================================
import numpy as np
import pandas as pd
import nltk
nltk.download('punkt')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.stem import PorterStemmer, SnowballStemmer
from nltk.corpus import stopwords
from nltk import download

from textblob import TextBlob

# Plots
# ==============================================================================
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
plt.rcParams['lines.linewidth'] = 1.5
%matplotlib inline

# Modeling and Forecasting
# ==============================================================================
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

from skforecast.ForecasterAutoreg import ForecasterAutoreg
from skforecast.ForecasterAutoregCustom import ForecasterAutoregCustom
#from skforecast.ForecasterAutoregMultiOutput import ForecasterAutoregMultiOutput
from skforecast.model_selection import grid_search_forecaster
from skforecast.model_selection import backtesting_forecaster

from joblib import dump, load

# Spark
# import SparkSession library 
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col, lower, concat_ws, regexp_replace, split, size, length, broadcast
from pyspark.sql.types import DoubleType, StructType, StructField, ArrayType, StringType, IntegerType, TimestampType
from pyspark.sql.types import *
import pyspark.sql.types as typ
import pyspark.sql.functions as fn
import pyspark.sql.functions as F


# Warnings configuration
# ==============================================================================
import warnings
# warnings.filterwarnings('ignore')

[nltk_data] Downloading package punkt to /home/hduser/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [3]:
# SparkContext 
sc.master

'local[*]'

In [4]:
spark = SparkSession.builder.appName("data_projectTweets").getOrCreate()

In [5]:
# try to creating a customer dataframe by declaring the schema and passing values, but didn't work


In [6]:
#schema = StructType().add("Ids","integer").add("Date","string").add("Flag", "string").add("User",'string').add("Text", "string")

In [7]:
# ****************************************************************************************************************
# *                                  Start with the Data                                                         *
# ****************************************************************************************************************

In [8]:
# Big headache and colapse, to read the dataset
# I was load the file from my hadoop (local)
# df = spark.read.csv('home/hduser/Documnets/CA2', header+True, inferSchema=True)

In [9]:
#Move the dataset fiel to HDFS path from my terminal:
#First check:  $fs -ls/user1
#Next move :   $hadoop fs-put./ProjectTweets.scv/user1
#Check again : $fs -ls/user1
#I will use direct from the path, to have one of five V' (Velocity) 


#folowiong you can see the two way to load the dataset, 
#    (command)+(file://)+(/path/)+(filename)
#I will use direct from the path, to have one of five V' (Velocity) 

Tweets_path = "/user1/ProjectTweets.csv"
#df = spark.read.csv("file:///home/hduser/Documents/CA2/ProjectTweets.csv", header=True, inferSchema = True)

In [10]:
Tweets_path

'/user1/ProjectTweets.csv'

In [11]:
#spark_df = spark.read.csv("file:///home/hduser/Documents/CA2/ProjectTweets.csv", header=True, inferSchema = True)

In [12]:
spark_df = spark.read.csv("/user1/ProjectTweets.csv", header=False, inferSchema=True)

In [13]:
column_names = ['_c0', 'Ids', 'Date', 'Flag', 'User', 'Text']

In [14]:
for i, column_name in enumerate(column_names):
    spark_df = spark_df.withColumnRenamed("_c" + str(i),column_name)

In [15]:
spark_df.show()

+---+----------+--------------------+--------+---------------+--------------------+
|_c0|       Ids|                Date|    Flag|           User|                Text|
+---+----------+--------------------+--------+---------------+--------------------+
|  0|1467810369|Mon Apr 06 22:19:...|NO_QUERY|_TheSpecialOne_|@switchfoot http:...|
|  1|1467810672|Mon Apr 06 22:19:...|NO_QUERY|  scotthamilton|is upset that he ...|
|  2|1467810917|Mon Apr 06 22:19:...|NO_QUERY|       mattycus|@Kenichan I dived...|
|  3|1467811184|Mon Apr 06 22:19:...|NO_QUERY|        ElleCTF|my whole body fee...|
|  4|1467811193|Mon Apr 06 22:19:...|NO_QUERY|         Karoli|@nationwideclass ...|
|  5|1467811372|Mon Apr 06 22:20:...|NO_QUERY|       joy_wolf|@Kwesidei not the...|
|  6|1467811592|Mon Apr 06 22:20:...|NO_QUERY|        mybirch|         Need a hug |
|  7|1467811594|Mon Apr 06 22:20:...|NO_QUERY|           coZZ|@LOLTrish hey  lo...|
|  8|1467811795|Mon Apr 06 22:20:...|NO_QUERY|2Hood4Hollywood|@Tatiana_K nop

In [16]:
# some excercises, it didn't work to me

In [17]:
#header = tweets.first()

#tweets = tweets.filter(lambda row: row != header) 
#tweets = tweets.map(lambda row: [int(elem) for elem in row.split(',')])

In [18]:
#tweets = sc.textFile("Tweets_path ")
#Header = tweets.first()

#tweets = tweets.filter(lambda row: row != header) 
#tweets = tweets.map(lambda row: [int(elem) for elem in row.split(',')])

In [19]:
# creating the schema for my DataFrame
# was one  error because i didn' import pyspark.sql.types as typ
# but next did it, the command worked, great!

In [20]:
#fields = [
#    *[
#        typ.StructField(h[1:-1], typ.IntegerType(), True)
#        for h in header.split(',')
#    ]
#]
#schema = typ.StructType(fields)

In [21]:
# First put the Titles to see better

In [22]:
#column_names  = ['_c0','Ids', 'Date', 'Flag', 'User', 'Text']    
#full_df = pd.read_csv('file:///home/hduser/Documents/CA2/ProjectTweets.csv', header=None, names=column_names)

In [23]:
# display the total number of rows data
total_rows = spark_df.count()


In [24]:
formatted_total_rows = "{:,}".format(total_rows)
print("Total Rows:", formatted_total_rows)

Total Rows: 1,600,000


In [25]:
# rename the file whit  row's title  ******* ******* ****** ******* ****** ********

In [27]:
spark_df.write.csv("file:///home/hduser/Documents/CA2/Title_pro_Tweets.csv", header=True)

In [28]:
# Drop unnecessary column  ************************************************************

In [29]:
column_to_drop = ['Flag', 'Unnamed: 0']
spark_df = spark_df.drop('Flag')

In [30]:
spark_df = spark_df.na.drop()

In [31]:
spark_df.show()

+---+----------+--------------------+---------------+--------------------+
|_c0|       Ids|                Date|           User|                Text|
+---+----------+--------------------+---------------+--------------------+
|  0|1467810369|Mon Apr 06 22:19:...|_TheSpecialOne_|@switchfoot http:...|
|  1|1467810672|Mon Apr 06 22:19:...|  scotthamilton|is upset that he ...|
|  2|1467810917|Mon Apr 06 22:19:...|       mattycus|@Kenichan I dived...|
|  3|1467811184|Mon Apr 06 22:19:...|        ElleCTF|my whole body fee...|
|  4|1467811193|Mon Apr 06 22:19:...|         Karoli|@nationwideclass ...|
|  5|1467811372|Mon Apr 06 22:20:...|       joy_wolf|@Kwesidei not the...|
|  6|1467811592|Mon Apr 06 22:20:...|        mybirch|         Need a hug |
|  7|1467811594|Mon Apr 06 22:20:...|           coZZ|@LOLTrish hey  lo...|
|  8|1467811795|Mon Apr 06 22:20:...|2Hood4Hollywood|@Tatiana_K nope t...|
|  9|1467812025|Mon Apr 06 22:20:...|        mimismo|@twittera que me ...|
| 10|1467812416|Mon Apr 0

In [32]:
#schema = StructType().add("_c0","integer").add("Ids","integer").add("Date","string").add("Flag", "string").add("User",'string').add("Text", "string")

In [33]:
# Creating a UDF to apply VADER sentiment analysis to a Text column

In [34]:
def analyze_sentiment(Text):
    sid = SentimentIntensityAnalyzer()
    sentiment = sid.polarity_scores(Text)
    return sentiment['compound']

In [35]:
#register the UDF

In [36]:
sentiment_udf = udf(analyze_sentiment, DoubleType())

In [37]:
#Apply sentiment

In [38]:
spark_df = spark_df.withColumn("sentiment", sentiment_udf(spark_df["Text"]))

In [40]:
print(spark_df.show())

+---+----------+--------------------+---------------+--------------------+---------+
|_c0|       Ids|                Date|           User|                Text|sentiment|
+---+----------+--------------------+---------------+--------------------+---------+
|  0|1467810369|Mon Apr 06 22:19:...|_TheSpecialOne_|@switchfoot http:...|  -0.0173|
|  1|1467810672|Mon Apr 06 22:19:...|  scotthamilton|is upset that he ...|    -0.75|
|  2|1467810917|Mon Apr 06 22:19:...|       mattycus|@Kenichan I dived...|   0.4939|
|  3|1467811184|Mon Apr 06 22:19:...|        ElleCTF|my whole body fee...|    -0.25|
|  4|1467811193|Mon Apr 06 22:19:...|         Karoli|@nationwideclass ...|  -0.6597|
|  5|1467811372|Mon Apr 06 22:20:...|       joy_wolf|@Kwesidei not the...|      0.0|
|  6|1467811592|Mon Apr 06 22:20:...|        mybirch|         Need a hug |   0.4767|
|  7|1467811594|Mon Apr 06 22:20:...|           coZZ|@LOLTrish hey  lo...|    0.745|
|  8|1467811795|Mon Apr 06 22:20:...|2Hood4Hollywood|@Tatiana_K n

In [41]:
#why i have Flag  again??
print(column_names[:6])

['_c0', 'Ids', 'Date', 'Flag', 'User', 'Text']


In [42]:
#to show

spark_df.drop('Flag').show()

+---+----------+--------------------+---------------+--------------------+---------+
|_c0|       Ids|                Date|           User|                Text|sentiment|
+---+----------+--------------------+---------------+--------------------+---------+
|  0|1467810369|Mon Apr 06 22:19:...|_TheSpecialOne_|@switchfoot http:...|  -0.0173|
|  1|1467810672|Mon Apr 06 22:19:...|  scotthamilton|is upset that he ...|    -0.75|
|  2|1467810917|Mon Apr 06 22:19:...|       mattycus|@Kenichan I dived...|   0.4939|
|  3|1467811184|Mon Apr 06 22:19:...|        ElleCTF|my whole body fee...|    -0.25|
|  4|1467811193|Mon Apr 06 22:19:...|         Karoli|@nationwideclass ...|  -0.6597|
|  5|1467811372|Mon Apr 06 22:20:...|       joy_wolf|@Kwesidei not the...|      0.0|
|  6|1467811592|Mon Apr 06 22:20:...|        mybirch|         Need a hug |   0.4767|
|  7|1467811594|Mon Apr 06 22:20:...|           coZZ|@LOLTrish hey  lo...|    0.745|
|  8|1467811795|Mon Apr 06 22:20:...|2Hood4Hollywood|@Tatiana_K n

In [43]:
print(spark_df.show())

+---+----------+--------------------+---------------+--------------------+---------+
|_c0|       Ids|                Date|           User|                Text|sentiment|
+---+----------+--------------------+---------------+--------------------+---------+
|  0|1467810369|Mon Apr 06 22:19:...|_TheSpecialOne_|@switchfoot http:...|  -0.0173|
|  1|1467810672|Mon Apr 06 22:19:...|  scotthamilton|is upset that he ...|    -0.75|
|  2|1467810917|Mon Apr 06 22:19:...|       mattycus|@Kenichan I dived...|   0.4939|
|  3|1467811184|Mon Apr 06 22:19:...|        ElleCTF|my whole body fee...|    -0.25|
|  4|1467811193|Mon Apr 06 22:19:...|         Karoli|@nationwideclass ...|  -0.6597|
|  5|1467811372|Mon Apr 06 22:20:...|       joy_wolf|@Kwesidei not the...|      0.0|
|  6|1467811592|Mon Apr 06 22:20:...|        mybirch|         Need a hug |   0.4767|
|  7|1467811594|Mon Apr 06 22:20:...|           coZZ|@LOLTrish hey  lo...|    0.745|
|  8|1467811795|Mon Apr 06 22:20:...|2Hood4Hollywood|@Tatiana_K n

In [44]:
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%>

In [45]:
# Creating my Dataframe (i did it in line 27 )

In [46]:
#        Basic Feature Extraction

In [47]:
#Header_df = spark.read.csv("file:///home/hduser/Documents/CA2/Header_Proj_Tweets.csv", header=True, inferSchema = True)

In [48]:
Tweets = spark_df

In [49]:
Tweets = Tweets.withColumn("word_count", size(split(Tweets["Text"], " ")))

In [50]:
# Number of Words  

In [51]:
#Tutorial did't work because is a Panda code: TypeError: 'Column' object is not callable

#Tweets['word_count'] = Tweets['Text'].apply(lambda x: len(str(x).split("")))
#Tweets[['Text','word_count']].head()

In [52]:
Tweets.select("Text", "word_count").show()

+--------------------+----------+
|                Text|word_count|
+--------------------+----------+
|@switchfoot http:...|        20|
|is upset that he ...|        22|
|@Kenichan I dived...|        19|
|my whole body fee...|        11|
|@nationwideclass ...|        22|
|@Kwesidei not the...|         6|
|         Need a hug |         4|
|@LOLTrish hey  lo...|        24|
|@Tatiana_K nope t...|         7|
|@twittera que me ...|         6|
|spring break in p...|         8|
|I just re-pierced...|         6|
|@caregiving I cou...|        21|
|@octolinz16 It it...|        16|
|@smarrison i woul...|        23|
|@iamjazzyfizzle I...|        20|
|Hollis' death sce...|        19|
|about to file taxes |         5|
|@LettyA ahh ive a...|        12|
|@FakerPattyPattz ...|        13|
+--------------------+----------+
only showing top 20 rows



In [53]:
# Number of Characters

In [54]:
#Tutorial did't work because is a Panda code:TypeError: 'Column' object is not callable
#Tweets['char_count'] = Tweets['Text'].str.len()
## this also includes spaces
#Tweets[['Text','char_count']].head()

In [55]:
Tweets = Tweets.withColumn("char_count", length(Tweets["Text"]))

In [56]:
Tweets.select("Text", "char_count").show()

+--------------------+----------+
|                Text|char_count|
+--------------------+----------+
|@switchfoot http:...|       115|
|is upset that he ...|       111|
|@Kenichan I dived...|        89|
|my whole body fee...|        47|
|@nationwideclass ...|       111|
|@Kwesidei not the...|        29|
|         Need a hug |        11|
|@LOLTrish hey  lo...|        99|
|@Tatiana_K nope t...|        36|
|@twittera que me ...|        25|
|spring break in p...|        43|
|I just re-pierced...|        26|
|@caregiving I cou...|        94|
|@octolinz16 It it...|        77|
|@smarrison i woul...|       117|
|@iamjazzyfizzle I...|       103|
|Hollis' death sce...|        93|
|about to file taxes |        20|
|@LettyA ahh ive a...|        64|
|@FakerPattyPattz ...|        79|
+--------------------+----------+
only showing top 20 rows



In [57]:
#Basic Pre-processing

In [58]:
Tweets = Tweets.withColumn("Text", concat_ws("", lower(col("Text"))))

In [59]:
#Removing Punctuation 

In [60]:
Tweets = Tweets.withColumn("Text", regexp_replace(col("Text"), r'[^\w\s]', ''))

In [61]:
Tweets.select("Text").show()

+--------------------+
|                Text|
+--------------------+
|switchfoot httptw...|
|is upset that he ...|
|kenichan i dived ...|
|my whole body fee...|
|nationwideclass n...|
|kwesidei not the ...|
|         need a hug |
|loltrish hey  lon...|
|tatiana_k nope th...|
|twittera que me m...|
|spring break in p...|
|i just repierced ...|
|caregiving i coul...|
|octolinz16 it it ...|
|smarrison i would...|
|iamjazzyfizzle i ...|
|hollis death scen...|
|about to file taxes |
|lettya ahh ive al...|
|fakerpattypattz o...|
+--------------------+
only showing top 20 rows



In [62]:
# Removal of Stop Words
#Load stop word once and broadcast it to all worker nodes

In [63]:

stop  = stopwords.words('english')
broadcasted_stopwords = spark.sparkContext.broadcast(stop)

In [64]:
#Define a funcion to remove stopwords

In [65]:
def remove_stopwords(text):
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in stop]
    return " ".join(filtered_words)

In [66]:
#Register the UDF

In [67]:
remove_stopwords_udf = udf(remove_stopwords, StringType())

In [68]:
#Apply the UDF

In [69]:
Tweets = Tweets.withColumn("Text", remove_stopwords_udf(Tweets["Text"]))

In [70]:
Tweets.select("Text").show()

+--------------------+
|                Text|
+--------------------+
|switchfoot httptw...|
|upset cant update...|
|kenichan dived ma...|
|whole body feels ...|
|nationwideclass b...|
| kwesidei whole crew|
|            need hug|
|loltrish hey long...|
|tatiana_k nope didnt|
|  twittera que muera|
|spring break plai...|
|      repierced ears|
|caregiving couldn...|
|octolinz16 counts...|
|smarrison wouldve...|
|iamjazzyfizzle wi...|
|hollis death scen...|
|          file taxes|
|lettya ahh ive al...|
|fakerpattypattz o...|
+--------------------+
only showing top 20 rows



In [71]:
#*****************************************************************************************************************
#*                                       T O K E N I Z A T I O N                                                 *
#*****************************************************************************************************************

In [72]:
# UDF for Tokenization

In [73]:
def tokenize_Tweets(Text):
    blob = TextBlob(Text)
    return [str(word) for word in blob.words]

In [74]:
tokenize_udf = udf(tokenize_Tweets, ArrayType(StringType()))

In [75]:
#Apply the UDF

In [76]:
spark_df = spark_df.withColumn('tokenized_Tweets', tokenize_udf(spark_df['Text']))

In [77]:
spark_df.show()

+---+----------+--------------------+---------------+--------------------+---------+--------------------+
|_c0|       Ids|                Date|           User|                Text|sentiment|    tokenized_Tweets|
+---+----------+--------------------+---------------+--------------------+---------+--------------------+
|  0|1467810369|Mon Apr 06 22:19:...|_TheSpecialOne_|@switchfoot http:...|  -0.0173|[switchfoot, http...|
|  1|1467810672|Mon Apr 06 22:19:...|  scotthamilton|is upset that he ...|    -0.75|[is, upset, that,...|
|  2|1467810917|Mon Apr 06 22:19:...|       mattycus|@Kenichan I dived...|   0.4939|[Kenichan, I, div...|
|  3|1467811184|Mon Apr 06 22:19:...|        ElleCTF|my whole body fee...|    -0.25|[my, whole, body,...|
|  4|1467811193|Mon Apr 06 22:19:...|         Karoli|@nationwideclass ...|  -0.6597|[nationwideclass,...|
|  5|1467811372|Mon Apr 06 22:20:...|       joy_wolf|@Kwesidei not the...|      0.0|[Kwesidei, not, t...|
|  6|1467811592|Mon Apr 06 22:20:...|        m

In [78]:
#****************************************************************************************************************
#*                                        S T R E M M I N G                                                     *
#****************************************************************************************************************

In [88]:
st = PorterStemmer()

In [89]:
# Define a Udf for Stremming

In [90]:
def stem_text(text):
    stemmer = SnowballStemmer("english")
    words = nltk.word_tokenize(Text)
    stemmer_words = [stemmer.stem(word) for word in words]
    return " ".join(stemmed_words)

In [91]:
# Register the UDF

In [92]:
strem_udf = udf(stem_text, StringType())

In [93]:
#Create a Spark session

In [95]:
spark = SparkSession.builder.appName("StemmingExample").getOrCreate()


In [96]:
#DataFrame 

In [97]:
Tweets_stemmed = spark_df.withColumn("Stemmed_Text", stem_udf(spark_df["Text"]))

NameError: name 'stem_udf' is not defined

In [ ]:
# Show te resulting DataFrame

In [ ]:
Tweets_stemmed.show(truncate=False)

In [ ]:
#Data Procesising in panda

In [ ]:
#df = df.withColumn("Text", lower(df["text"]))
#df = df.withColumn("Text", regexp_replace(df["text"], "[^a-z0-9\\s]", ""))

In [ ]:
df = df.withColumn("Text", lower(df["text"]))
df = df.withColumn("Text", regexp_replace(df["text"], "[^a-z0-9\\s]", ""))

In [ ]:
rows_df.shape